### Script de modelagem direta da bacia de reconcavo - dados grav da ANP:

In [1]:
# importando pacotes necessarios:
import numpy as np # pacote numerico
import pylab as py 
import pandas as pd
#from numpy.linalg import inv # pacote de algebra linear
import matplotlib.pyplot as plt # pacote de plotagem
#import pandas as pd # pacode de tratamento de dados 
#from mpl_toolkits.basemap import Basemap
#from matplotlib.patches import Polygon
#from scipy.interpolate import Rbf # radial basis functions technique
#para plotar as figuras nas celulas do jupyter notebook:
from matplotlib.path import Path
from matplotlib.patches import PathPatch
#%matplotlib inline
#%matplotlib qt
import utm
#plt.plot ([1,2])

### Importando os pacotes com as nossas funcoes:

In [2]:
from modules.iplots import pick_points, model_masses # para fazer o plot interativo
from modules.gravity import grav2D_anom, g_sphere # funcao que calcula dados de gravidade (rao 1994)

### Leitura dos dados na seção obtidos com a continuação:

In [3]:
Lon, Lat, Boug_residual= np.loadtxt('bouguer_recon_perfil_Cont4000_cut.txt',
                                             skiprows=1, usecols=(0,1,2), unpack=True)


In [4]:
Lon

array([-38.84      , -38.83925806, -38.83851613, -38.83777419,
       -38.83703226, -38.83629032, -38.83554839, -38.83480645,
       -38.83406452, -38.83332258, -38.83258065, -38.83183871,
       -38.83109677, -38.83035484, -38.8296129 , -38.82887097,
       -38.82812903, -38.8273871 , -38.82664516, -38.82590323,
       -38.82516129, -38.82441935, -38.82367742, -38.82293548,
       -38.82219355, -38.82145161, -38.82070968, -38.81996774,
       -38.81922581, -38.81848387, -38.81774194, -38.817     ,
       -38.81625806, -38.81551613, -38.81477419, -38.81403226,
       -38.81329032, -38.81254839, -38.81180645, -38.81106452,
       -38.81032258, -38.80958065, -38.80883871, -38.80809677,
       -38.80735484, -38.8066129 , -38.80587097, -38.80512903,
       -38.8043871 , -38.80364516, -38.80290323, -38.80216129,
       -38.80141935, -38.80067742, -38.79993548, -38.79919355,
       -38.79845161, -38.79770968, -38.79696774, -38.79622581,
       -38.79548387, -38.79474194, -38.794     , -38.79

In [5]:
# lat lon do banco de dados Oasis computados pela funcao utm:
npo = len(Lon)
print ('numero de dados=', npo)
UTMe = np.zeros(npo)
UTMn = np.zeros(npo)
#ZONE=np.zeros(npo)

for i in range(npo):
    UTMe[i], UTMn[i], _ = utm.from_latlon(Lat[i], Lon[i],force_zone_number=24) # output = UTM em m


numero de dados= 931


In [6]:
UTMe

array([517381.36666179, 517461.95687258, 517542.5470023 , 517623.13705096,
       517703.72701857, 517784.31690515, 517864.90671069, 517945.49643522,
       518026.08607874, 518106.67564127, 518187.26512281, 518267.85452338,
       518348.44384299, 518429.03308165, 518509.62223937, 518590.21131616,
       518670.80031203, 518751.389227  , 518831.97806107, 518912.56681425,
       518993.15548656, 519073.74407801, 519154.3325886 , 519234.92101836,
       519315.50936728, 519396.09763538, 519476.68582268, 519557.27392918,
       519637.86195489, 519718.44989983, 519799.03776401, 519879.62554743,
       519960.2132501 , 520040.80087205, 520121.38841328, 520201.97587379,
       520282.56325361, 520363.15055274, 520443.7377712 , 520524.32490899,
       520604.91196612, 520685.49894261, 520766.08583847, 520846.67265371,
       520927.25938833, 521007.84604236, 521088.4326158 , 521169.01910866,
       521249.60552095, 521330.19185269, 521410.77810388, 521491.36427454,
       521571.95036468, 5

## Plots

Continuação 4000m

In [24]:
# Plot dos dados para verificação:
plt.figure(figsize=(15,7), facecolor='w')

p1 = plt.scatter(UTMe,UTMn,s=5.0,c=Boug_residual,marker='*', cmap=plt.cm.jet)
cbar = plt.colorbar(p1, aspect=30, fraction = 0.03, orientation="vertical")
cbar.set_label('Residual Bouguer Anomaly $ (mGal) $',fontsize=15, rotation = 90)
#plt.plot(x_utm, gz,'*r')
plt.title ('UPWARD CONTINUATION (4000m)', fontsize = 16)
plt.xlabel ('Easting ($m$)', fontsize = 14)
plt.ylabel ('Northing ($m$)', fontsize = 14)
plt.grid()
#plt.savefig('Bouguer_profile_continuation4000.png', dpi=300, transparent=True, bbox_inches="tight" )
plt.show()

### Definicao da area do modelo de bacia sedimentar:

In [8]:
area= []
xmin = np.min(UTMe) # em m
xmax = np.max(UTMe) # em m
zmin =  0.0 # Nivel medio dos mares (verificar) #np.min( (elev) ) #-100.0
zmax =4000.0 #adaptado para metros (vindo da analise dos dados sismicos) !
area = [xmin, xmax, zmin, zmax]
print(area)


[517381.36666178907, 592296.7037156699, 0.0, 4000.0]


### Criacao do modelo de bacia atraves dos clicks na area de plot: 

In [28]:
%matplotlib qt 
axes = plt.figure().add_subplot(1,1,1)
xv,zv = pick_points(area, axes, marker='o', color='k', size=8, xy2ne=False)

### calculo da anomalia de gravidade produzido pelo modelo de bacia (artigo do Rao et. al., 1994):

### <center> Para usar Rao, 1994: coordenadas em Km e densidade em g/cm³

In [10]:
# info da propriedade fisica:
delta_rho = -0.2 #(g/cm³)# contraste de Densidade do arenito em relacao ao embasamento local no topo da bacia!
beta =10.57
# convertendo para SI(kg/m³):
rho_ref = 3.0 # densidade do embasamento (checar!)
rho = delta_rho + rho_ref
#rho = rho * 1000.0 
print('densidade no topo da bacia em g/cm³=', rho)

densidade no topo da bacia em g/cm³= 2.8


In [25]:
#Criacao de variaveis necessarias para o calculo da anomalia:
nv = np.size(xv)
#print nv
nper = np.size(UTMe)
#print nper
# calculo da anomalia grav atraves de rao1994:
xd = np.zeros( (nv,) )
zd = np.zeros( (nv,) )
grav = np.zeros( (nper,) )
for i in range(nper):
    for j in range(nv):
        xd[j] = ( xv[j] - UTMe[i])/1000.0 # em Km
        zd[j] = (zv[j] - 0)/1000.0 # em Km
    # chamando a funcao do rao 1994:
    grav[i] = grav2D_anom(xd,zd,delta_rho,beta)

In [26]:
#np.size (xv)

In [27]:
# computing hyperbolic density function
###################################################

z = np.linspace( np.amin(zv), np.amax(zv), np.size(zv) ) # em m
rhoz = (delta_rho)*beta**2/(beta + (z/100.0) )**2 # divisao por 100 = conversao para cm
#rhoz = rhoz + 2.670

In [14]:
# save figure and a file for reproduction:
A = np.zeros( (len(z),2) )
A[:,0] = z
A[:,1] = rhoz

#caminho = 'figs/densityCONTINUATION_intrusion5'
# save text file for ploting issues:
#np.savetxt(caminho+'.txt', A, header=' Z , rhoZ', delimiter=' ' )

In [15]:
# plot of the density distribution:
plt.close('all')
fig = plt.figure(figsize=(7,7))

# definition of fontsize:
fs = 15 
# invert axis
plt.gca().invert_yaxis()
plt.gca().invert_xaxis()

# set labelsize 
plt.tick_params(axis='y', labelsize=fs-2)
plt.tick_params(axis='x', labelsize=fs-2,labelbottom=True,labeltop=False)

plt.text(rhoz[0]+0.01, z[0]+0.148, str( format(rhoz[0],'.3f') ), style='italic',
        bbox={'facecolor':'red', 'alpha':0.5, 'pad':10})

plt.plot(rhoz,z,'k-',linewidth=2.0)
plt.plot(rhoz[0],z[0],'ro',linewidth=2.0)
plt.grid()
plt.xlabel('Contraste de densidade $(g/cm^3)$',fontsize=fs)
plt.ylabel('Profundidade $(m)$',fontsize=fs)
plt.axis([np.max(rhoz)+0.05, np.min(rhoz)-0.02, np.max(z)+0.1, np.min(z)-0.1])
#plt.savefig(caminho+'.png', dpi=300, transparent=True, bbox_inches="tight" )
plt.show()
#############################################################################################################

In [16]:
# get the number of elements of xv (number of vertices of the polygon)
n = np.size(xv)     
# create new working arrays for the vertices of a closed polygon:
x = np.zeros( (n+1,) )  
z = np.zeros( (n+1,) ) 
x[0:n] = xv
z[0:n] = zv
# GAMBIARRA PARA PLOT DE CORES ASSSOCIADOOS À BACIA MODELADA (PENSAR EM ALGO MILHÓ) 
x[n:n+1] = min(xv)
z[n:n+1] = np.min(zv)

In [ ]:
# save fig and a txt file for reproduction:
#caminho = "figs/modelCONTINUATION_intrusion_reconcavo1"

B = np.zeros( (len(z),2) )
B[:,0] = x
B[:,1] = z

# save text file for ploting issues:
#np.savetxt(caminho+'.txt', B, header=' Easting(m) , depth(m)', delimiter=' ' )

In [17]:
# and the modeled basin:
plt.close('all')
fig = plt.figure(figsize=(14,8))
path = Path(np.array([x,z]).transpose())
patch = PathPatch(path, facecolor='none')

plt.gca().add_patch(patch) 
plt.plot(xv,zv,'k-o') #DEVE SER XV E ZV REALMENTE!!!

# plot the last and the first corner to close up the polygon:
fs = 18 # font size for the label
plt.gca().invert_yaxis()
plt.xlabel(r'UTM E $(m)$',fontsize=fs)
plt.ylabel(r'Profundidade $(m)$',fontsize=fs)
plt.xlim([np.min(xv), np.max(xv)])

##################################################################
# plot the density variation together with the basin:
##################################################################

im = plt.imshow(rhoz.reshape(np.size(zv),1),  cmap=plt.cm.Wistia,interpolation="bicubic",
                origin='lower',extent=[min(xv), max(xv), min(zv), max(zv)],aspect="auto", clip_path=patch, clip_on=True)
#im.set_clip_path(patch)

# OBS:::::: para reverter a escala de cores, basta um simples "_r" ! sucesso de vida!!!!
plt.gca().invert_yaxis()
cbar = plt.colorbar()
cbar.ax.set_ylabel(r'Contraste de densidade $g/cm^3$', fontsize=fs)

#plt.savefig(caminho+'.png', dpi=300, transparent=True, bbox_inches="tight" )
plt.show()

### Plot do modelo atual + os dados reais para comparação:

In [ ]:
# save fig and a txt file for reproduction:
#caminho = "figs/datafitCONTINUATION_intrusion5"

C = np.zeros( (len(UTMe),3) )
C[:,0] = UTMe
C[:,1] = grav
C[:,2] = Boug_residual

# save text file for ploting issues:
#np.savetxt(caminho+'.txt', C, header=' Easting(m) Gpred(mGal) Gobs(mGal)', delimiter=' ' )

In [18]:
plt.plot(UTMe, grav,'.r', label='dado predito')

In [19]:
plt.close('all')
plt.plot(UTMe, Boug_residual,'.k', label='dado observado')
plt.plot(UTMe, grav,'.r', label='dado predito')
plt.legend()
plt.grid()
plt.xlabel('Longitude (m)', fontsize = 14)
plt.ylabel('Anomalia Bouguer (mGal)', fontsize = 14)
#plt.savefig(caminho+'.png', dpi=300, transparent=True, bbox_inches="tight" )
plt.show()

# Criacao de um branch para nao perder clicks promissores!!!!!!

In [ ]:
## criacao do branch --- variaveis guardadas para nao perder cliques promissores!
### SOMENTE RODAR ESSA CELULA EM CASO DE MELHORIAS SIGNFICATIVAS NOS CLICKS!!!!
x_branch = np.copy(xv)
y_branch = np.copy(zv)
gz_branch = np.copy(grav)

### TODO:: Gerar um arquivo com as variaveis branch para backup de clicks promissoes!

In [ ]:
# ---- comparacao entre o dado branch vs o ultimo click-----:
plt.plot(UTMe, grav,'.k', label='observed data')
plt.plot(UTMe, gz_branch,'.m', label='branch data')
plt.legend()
plt.grid()
plt.title ('Gravity Data (mGal)', fontsize = 16)
plt.xlabel ('UTM x(m)', fontsize = 14)
plt.ylabel ('Gravity Anomaly (mGal)', fontsize = 14)
plt.show()